In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [2]:
import numpy as np
import transformers
import pandas as pd
# import matplotlib.pyplot as plt
import torch
# import seaborn as sns
from pyspark.sql.types import ArrayType, IntegerType

from torch.utils.data import Dataset
import torch.nn.utils.rnn as rnn_utils
# import matplotlib.pyplot as plt
import time
from pyspark.sql.functions import *
# import seaborn as sns
import statsmodels.api as sm
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

from pyspark.ml.feature import StringIndexer, VectorIndexer, StringIndexerModel, IndexToString
from pyspark.sql import SparkSession
#conf = SparkConf().setAppName('Elephas_App').setMaster('local[8]')
#sc = SparkContext(conf=conf)


i:\CODE\PROJECT_COURSE\BIGDATA\FakeNewDT_Vietnamese\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Acer\AppData\Local\Temp\ipykernel_21464\524260765.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [4]:
sqlContext = spark.sparkContext
sqlContext

<SparkContext master=local appName=kafka-example>

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, FloatType

# Define the schema for your DataFrame
schema = StructType([
    StructField("post_message", StringType(), True),
])

In [6]:

# Specify the Kafka parameters
kafka_params = {
    "kafka.bootstrap.servers": "localhost:9092",  # Change this to your Kafka broker
    "subscribe": "FakeNewDT",               # Change this to your Kafka topic
    "startingOffsets": "earliest"
}

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

In [8]:
# Pytorch's nn module has lots of useful feature
import torch.nn as nn

class LSTMNet(nn.Module):

    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout):

        super(LSTMNet,self).__init__()

        # Embedding layer converts integer sequences to vector sequences
        self.embedding = nn.Embedding(vocab_size,embedding_dim)

        # LSTM layer process the vector sequences
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout,
                            batch_first = True
                           )

        # Dense layer to predict
        self.fc = nn.Linear(hidden_dim * 2,output_dim)
        # Prediction activation function
        self.sigmoid = nn.Sigmoid()


    def forward(self,text,text_lengths):
        embedded = self.embedding(text).to(device)

        # Thanks to packing, LSTM don't see padding tokens
        # and this makes our model better
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)

        packed_output,(hidden_state,cell_state) = self.lstm(packed_embedded)

        # Concatenating the final forward and backward hidden states
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)

        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.sigmoid(dense_outputs)

        return outputs


In [9]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
vocab_size = tokenizer.vocab_size
embedding_dim = 300
hidden_dim = 64
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2


In [10]:
load_model = LSTMNet(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout)

In [11]:
device = torch.device("cpu")


In [12]:
load_model = load_model.to(device)


In [13]:
load_model.load_state_dict(torch.load('../model/model_0.9178659543395042.pth',map_location=torch.device('cpu')))


<All keys matched successfully>

In [14]:
load_model.eval()


LSTMNet(
  (embedding): Embedding(64000, 300)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [15]:
def stream_predict(text, tokenizer, model, device):
    tokens = tokenizer.tokenize(text)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)

    ids = ids.to(device)

    # Get the length of the input sequence
    length = torch.tensor(ids.shape[1], dtype=torch.long).unsqueeze(0)

    # Evaluate the model on the input text
    with torch.no_grad():
        model.eval()
        predictions = model(ids, length)


    binary_predictions = torch.round(predictions).cpu().numpy()

    return binary_predictions[0][0]

In [16]:
kafka_stream_df = (
    spark.read.format("kafka")
    .option("kafka.bootstrap.servers", kafka_params["kafka.bootstrap.servers"])
    .option("subscribe", kafka_params["subscribe"])
    .option("startingOffsets", kafka_params["startingOffsets"])
    .load()
    .selectExpr("CAST(value AS STRING)")
    .select(from_json("value", schema).alias("data"))
    .select("data.*")
)
# Apply the UDF to create a new column 'predictions'
data = kafka_stream_df.toPandas()['post_message']
data = data.head(10)

In [17]:
len(data)

10

In [18]:
# Tạo một DataFrame rỗng
df = pd.DataFrame(columns=['post_message', 'predictions'])

In [19]:
# from time import sleep
# from IPython.display import display, clear_output
# for i in range(len(data)):
#     result = stream_predict(data[i], tokenizer, load_model, device)
#     df = df.append({'post_message': data[i], 'predictions': result}, ignore_index=True)
#     print(df)
#     sleep(3)
#     clear_output(wait=True)
# # In ra DataFrame

import pandas as pd
from time import sleep
from IPython.display import display, clear_output

# Tạo DataFrame trống
df = pd.DataFrame(columns=['post_message', 'predictions'])
sent = []
pred = []
for i in range(len(data)):
    result = stream_predict(data[i], tokenizer, load_model, device)
    # df = df.append({'post_message': data[i], 'predictions': result}, ignore_index=True)
    sent.append(data[i])
    pred.append(result)
    print(sent)
    print(pred)
    sleep(3)
    clear_output(wait=True)


['Vụ Trường quốc_tế Singapore thu 80% học_phí khi dạy online : Không được phép khi chưa thoả_thuận được với phụ_huynh', 'Dự báo thời tiết hôm nay 5/5: Nắng nóng gia tăng ở Bắc Bộ và Trung Bộ, có nơi trên 38 độ C', 'Mấy chú Ba Tàu bán bánh bao và mỳ vằn thắn chơi Bà đầm già quả này vừa mất tiền oan:\r\n\r\nhttps://zingnews.vn/anh-bo-20-trieu-usd-mua-xet-nghiem-tu-tq-nhung-khong-dung-duoc-post1074158.html\r\n\r\n Lại thêm nỗi lo dân mất mạng..\r\n\r\nhttps://vnexpress.net/anh-thua-dau-vi-danh-cuoc-vao-kit-xet-nghiem-trung-quoc-4085956.html', '️🎉 KHAI_TRƯƠNG TUYẾN CÁP VƯỢT BIỂN CÓ TRỤ CÁP TREO CAO NHẤT THẾ_GIỚI Ở Hải_Phòng , Phó Thủ_tướng Thường_trực Trương_Hoà_Bình đã tới dự : 🏩 Lễ động_thổ khách_sạn 5 sao và khai_trương bãi biển nhân_tạo Dragon tại Tổ_hợp du_lịch Quốc_tế Đồi_Rồng ở quận Đồ_Sơn ; 🏖 Lễ khánh_thành tổ_hợp nghỉ_dưỡng cao_cấp Flamingo_Cát_Bà do Flamingo đầu_tư .🚟 Lễ khai_trương tuyến cáp treo Cát Hải-Phù Long do Sun_Group đầu_tư ( rút ngắn thời gian di chuyển tới Cát Ba